# TensorFlow

To import tensorflow, first install it in your system using pip install --upgrade tensorflow


In [1]:
import tensorflow as tf

# create a reset graph function so that we can reset a shell or a pipeline without overwriting the graph

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x=tf.Variable(3, name='x')
y=tf.Variable(5,name='y')
f=x*x*y + y + 5

In [3]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result=f.eval()
    


In [5]:
result

55

In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

In [7]:
result

55

# Linear Regression

In [8]:
import numpy as np
from sklearn.datasets import fetch_california_housing


Adding bias to the dataset

In [9]:
housing = fetch_california_housing()
m,n=housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

In [10]:
m #rows or instances

20640

In [11]:
n # columns or feaatures

8

In [12]:
housing

{'DESCR': 'California housing dataset.\n\nThe original database is available from StatLib\n\n    http://lib.stat.cmu.edu/datasets/\n\nThe data contains 20,640 observations on 9 variables.\n\nThis dataset contains the average house value as target variable\nand the following input variables (features): average income,\nhousing average age, average rooms, average bedrooms, population,\naverage occupation, latitude, and longitude in that order.\n\nReferences\n----------\n\nPace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,\nStatistics and Probability Letters, 33 (1997) 291-297.\n\n',
 'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.     

In [13]:
housing_data_plus_bias


array([[   1.        ,    8.3252    ,   41.        , ...,    2.55555556,
          37.88      , -122.23      ],
       [   1.        ,    8.3014    ,   21.        , ...,    2.10984183,
          37.86      , -122.22      ],
       [   1.        ,    7.2574    ,   52.        , ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.        ,    1.7       ,   17.        , ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.        ,    1.8672    ,   18.        , ...,    2.12320917,
          39.43      , -121.32      ],
       [   1.        ,    2.3886    ,   16.        , ...,    2.61698113,
          39.37      , -121.24      ]])

In [14]:
X=tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y=tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT=tf.transpose(X)
theta= tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

In [15]:
with tf.Session() as sess:
    theta_value = theta.eval()
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

standardising the dataset using sklearn's preprocessing module

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_bias_data = np.c_[np.ones((m,1)),scaled_housing_data]

In [17]:
print(scaled_housing_bias_data.mean(axis=0))
print(scaled_housing_bias_data.mean(axis=1))
print(scaled_housing_bias_data.mean())
print(scaled_housing_bias_data.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [18]:
scaled_housing_bias_data.std()

0.9938079899999067

In [19]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_bias_data, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.53737885
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293705


In [20]:
best_theta

array([[ 2.06855226e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44081801e-04],
       [-3.91945131e-02],
       [-8.61356556e-01],
       [-8.23479712e-01]], dtype=float32)

In [21]:
A=tf.placeholder(tf.float32, shape=(None,3))
B = 5+A
with tf.Session() as sees:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
print (B_val_1)

[[6. 7. 8.]]


In [22]:
reset_graph()

X=tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y=tf.placeholder(tf.float32, shape=(None,1), name='y')


theta=tf.Variable(tf.random_uniform([n+1,1],-1,1,seed=23),name='theta')
y_pred=tf.matmul(X,theta,name='Predictor')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [23]:
n_epoch=10
batch_size=100
n_batches=int(np.ceil(m/batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch*n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)  
    X_batch = scaled_housing_bias_data[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [24]:
best_theta

array([[ 2.0714476 ],
       [ 0.8462012 ],
       [ 0.11558535],
       [-0.26835832],
       [ 0.32982782],
       [ 0.00608358],
       [ 0.07052915],
       [-0.87988573],
       [-0.8634251 ]], dtype=float32)